<a href="https://www.kaggle.com/code/foocheechuan/amex-default-prediction-improvement?scriptVersionId=163185103" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amexfeather/test_data_f32.ftr
/kaggle/input/amexfeather/train_data.ftr
/kaggle/input/amexfeather/train_data_f32.ftr
/kaggle/input/amexfeather/test_data.ftr
/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet
/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


# This notebook is an improvement to the baseline model 
https://www.kaggle.com/code/foocheechuan/amexdefaultprediction

<a id="table-of-content"></a>
# Table of Content
### [1. Setup](#setup)
- [Import Libraries](#import-libraries)
- [Dataset](#dataset)

### [2. Handling Missing Values](#missing)
- [Removes columns with >50% missing values](#50%-missing)
- [Simple Imputer](#simple-imputer)

### [Go to end](#end)

# Setup
<a id="setup"></a>

# Import Libraries
<a id="import-libraries"></a>

In [2]:
# data preparation
import pandas as pd
import numpy as np

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  
from sklearn.metrics import precision_score                         
from sklearn.metrics import recall_score
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier, log_evaluation


# Dataset
<a id="dataset"></a>

- The original dataset provided in csv is too large (50GB)
- The data cannot fit into memory
- [Amex-Feather-Dataset](#https://www.kaggle.com/datasets/munumbutt/amexfeather) provided by [@munum](#https://www.kaggle.com/munumbutt) is a [feather file](#https://arrow.apache.org/docs/python/feather.html) that has smaller size than an equivalent csv file

# Read Data

In [3]:
# train.shape = (5531451,190)
train = pd.read_parquet('/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet')

# use small dataset to prevent insufficient memory
train_small = train.head(100)

In [20]:
# Aggregate the rows by customer_ID reduces the number of rows and add many features
train_num_agg = train_small.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])

# Add aggregation method to the column names
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
train_num_agg.reset_index(inplace = True)

# Feature engineering for categorical columns
train_cat_agg = train_small.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
train_cat_agg.reset_index(inplace = True)


train_cat_agg

,customer_ID,B_30_count,B_30_last,B_30_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,D_114_last,D_114_nunique,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,13,0,1,13,2,1,13,1,1,...,1,13,0,1,13,-1,1,13,6,1
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,13,0,1,13,2,1,13,1,1,...,1,13,0,1,13,-1,1,13,6,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,13,0,1,13,1,1,13,1,2,...,1,13,2,1,13,-1,1,13,6,1
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,13,0,1,13,2,1,13,1,1,...,1,13,0,1,13,-1,1,13,3,3
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,13,0,1,13,1,2,13,1,1,...,1,13,0,1,13,1,1,13,6,1
5,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...,13,0,1,13,2,2,13,1,1,...,1,13,2,1,13,-1,1,13,6,1
6,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,13,0,3,13,3,3,13,0,2,...,1,13,2,2,13,-1,1,13,3,3
7,0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4c...,9,0,1,9,5,2,9,1,1,...,1,9,2,1,9,-1,1,9,5,1


In [ ]:
# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    train = pd.read_parquet('/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet')
    
    # removes id and time from training set
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')

In [ ]:
%%time
train_data = pd.read_feather('../input/amexfeather/train_data.ftr')
test_data = pd.read_feather('../input/amexfeather/test_data.ftr')
train=train_data.groupby('customer_ID').tail(1)
train=train.set_index(['customer_ID'])
# There are multiple transactions. Lets take only the latest transaction from each customer.
test=test_data.groupby('customer_ID').tail(1)
test=test.set_index(['customer_ID'])
del train_data
del test_data

In [ ]:
train.info(max_cols=200, show_counts=True)

Insights (Train):
- There are many columns with missing values: Some with a lot missing values, some not much. So we wouldn't drop all columns with missing values.
- We wouldn't know whether the column is important because the column names are anonymous. Hence, we need to find statistical proof to justify dropping the columns.

# Handling Missing Values
<a id="missing"></a>

In [ ]:
# Calculate how many % instances are missing in each column
null_percent = ((train.isnull().sum())/train.shape[0]).tolist()
# null_percent

## Removes columns with >50% missing values
<a id="50%-missing"></a>

In [ ]:
# n where n is the nth column in the data frame that has more than 50% instances as null
null_list = []
for i in range(0,len(null_percent)):
    if null_percent[i]>=0.5:
        null_list.append(i)
null_list # return a list that contains which column has >50% missing values
del null_percent

In [ ]:
# Drop columns that have .50% missing values
train_drop = train.drop(train.columns[null_list],axis=1)
test_drop = test.drop(test.columns[null_list],axis=1)
del train
del test
del null_list

When more than half of the columns are missing, probably the column is not going to be useful regardless of what imputing method you use unless it tells something about the column like (missing = no account) or something. Which it is impossible for us to know unless you can speak to the data owner.

Hence, we dropped 30 columns (>50% missing values) and now we have only 161 columns left.

In [ ]:
train_drop.info(max_cols=200, show_counts=True)

Let's use simple imputer to impute the other columns with missing values

# Preprocessing Data

In [ ]:
df_drop = train_drop.drop(columns=["target",'S_2'], axis=1)
test_final = test_drop.drop(columns=['S_2'], axis=1)
# X_train.info(max_cols=200, show_counts=True)

In [ ]:
y = train_drop["target"]
del train_drop
del test_drop

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_drop, y, test_size = 0.1)

In [ ]:
clf = LGBMClassifier(n_estimators=1200,
                          learning_rate=0.03, reg_lambda=50,
                          min_child_samples=2400,
                          num_leaves=95,
                          colsample_bytree=0.19,
                          max_bins=511, random_state=1)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
#Test the model
y_predict=clf.predict(X_test)
print('LGBM Classifier Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predict)))
# Achieved 88.4% accuracy

In [ ]:
#Predict probabilities of default
y_test_predict=clf.predict_proba(test_final)

In [ ]:
a = pd.DataFrame({"prediction":y_predict})

In [ ]:
a['prediction'].value_counts()

In [ ]:
y_test_predict

In [ ]:
#Retrieve the probability of default
y_predict_final=y_test_predict[:,-1]

# Merge the prediction and customer_ID into submission dataframe
submission = pd.DataFrame({"customer_ID":test_final.index,"prediction":y_predict_final})

submission.to_csv('submission.csv', index=False)

## Work in Progress
- Model Optimization
- Try different aggregation method
- Stratified KFold validation
- Feature Engineering

# References
<a id="references"></a>
- [AMEX EDA which makes sense](#https://www.kaggle.com/code/ambrosm/amex-eda-which-makes-sense)
- [AMEX - Light GBM](#https://www.kaggle.com/code/lixinqi98/amex-lightgbm/notebook)
- [AMEX Default Prediction - EDA & Prediction](#https://www.kaggle.com/code/aryanml007/amex-default-prediction-eda-prediction)

# [Back](#table-of-content)
<a id="end"></a>